# Icsstrive.

## Importación de librerías.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import country_converter as coco
import matplotlib.pyplot as plt
import re
import ast
from urllib.parse import urlparse

## Importación e información de dataframes.

In [2]:
df = pd.read_csv('./data/ICSSTRIVE.csv')
df

,data_source_link_url,description,date,locations,estimated_cost,victims,type_of_malware,threat_source,industries,impacts,references
0,https://icsstrive.com/incident/operational-imp...,ALPS' North American production operations and...,"September 10, 2023","['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...",No Malware identified,"[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']",OT IT,[{'title': 'Cyber Incident Victim: Alps Alpine...
1,https://icsstrive.com/incident/framework-lapto...,"San Francisco, California-based Framework, the...","January 9, 2024",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No Malware identified,No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...
2,https://icsstrive.com/incident/clothing-provid...,"After suffering a cyberattack December 13, V.F...","December 13, 2023",['United States'],"Cost not known, 35.5 million customers persona...","[{'title': 'V.F. Corporation', 'link_url': 'ht...",No Malware identified,No threat source identified,['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H..."
3,https://icsstrive.com/incident/hackers-accesse...,Toyota Financial Services (TFS) is warning cus...,"December 11, 2023",['Germany'],No cost values disclosed.,"[{'title': 'Toyota Financial Services, subs. o...",No Malware identified,No threat source identified,['Automotive'],IT Privacy,[{'title': 'Toyota warns customers of data bre...
4,https://icsstrive.com/incident/chinese-identif...,Chinese hackers are positioning themselves ins...,"December 11, 2023",['United States'],No cost values disclosed.,"[{'title': 'Unidentified Oil & Gas Pipeline', ...",No Malware identified,"[{'title': 'Volt Typhoon', 'link_url': 'https:...","['Oil & Gas', 'Water and Waste Water']",IT,[{'title': 'Chinese hackers allegedly target U...
...,...,...,...,...,...,...,...,...,...,...,...
1101,https://icsstrive.com/incident/ransomware-atta...,Bassett Furniture was forced to shut down its ...,"July 10, 2024",['United States'],"unknown, manufacturing shut down","[{'title': 'Bassett Furniture', 'link_url': 'h...",No Malware identified,No threat source identified,['Manufacturing'],OT IT,[{'title': 'Bassett Furniture reports signific...
1102,https://icsstrive.com/incident/rhysida-ransomw...,"The government of Columbus, Ohio shut down a l...","July 18, 2024",['United States'],$1.9M ransom demand; IT systems shut down,"[{'title': 'Ohio State Government', 'link_url'...",No Malware identified,"[{'title': 'Rhysida Ransomware Group', 'link_u...",['Government'],IT,[{'title': 'Columbus says it thwarted overseas...
1103,https://icsstrive.com/incident/threat-actor-ta...,"A threat actor known as ""Sticky Werewolf"" is u...","June 7, 2024","['Belarus', 'Russia']",No cost values disclosed.,"[{'title': 'Russian Aviation Industry', 'link_...","[{'title': 'Phishing Attack', 'link_url': 'htt...","[{'title': 'Sticky Werewolf', 'link_url': 'htt...","['Transportation (Includes Logistics, Shipping...",IT,[{'title': 'STICKY WEREWOLF TARGETS THE AVIATI...
1104,https://icsstrive.com/incident/wichita-city-ra...,"The cyberattack at the city of Wichita, Kansas...","May 3, 2024",['United States'],CIty services down for >1 month,"[{'title': 'City of Wichita', 'link_url': 'htt...",No Malware identified,"[{'title': 'LockBit', 'link_url': 'https://ics...","['Government', 'Water and Waste Water']",IT Privacy,[{'title': 'City of Wichita water bill payment...


In [3]:
df.describe(include='all')

,data_source_link_url,description,date,locations,estimated_cost,victims,type_of_malware,threat_source,industries,impacts,references
count,1106,1106,1106,1106,1106,1106,1106,1106,1106,723,1106
unique,1058,860,672,135,355,790,60,97,91,12,820
top,https://icsstrive.com/incident/mumu-infection-...,WIZARD SPIDER is a sophisticated eCrime group ...,"January 1, 2003",['United States'],No cost values disclosed.,No victims identified,No Malware identified,No threat source identified,"['Transportation (Includes Logistics, Shipping...",IT,No references identified.
freq,2,239,34,413,299,260,877,818,159,443,282


In [4]:
df.shape

(1106, 11)

In [5]:
df.dtypes

data_source_link_url    object
description             object
date                    object
locations               object
estimated_cost          object
victims                 object
type_of_malware         object
threat_source           object
industries              object
impacts                 object
references              object
dtype: object

In [6]:
df.columns

Index(['data_source_link_url', 'description', 'date', 'locations',
       'estimated_cost', 'victims', 'type_of_malware', 'threat_source',
       'industries', 'impacts', 'references'],
      dtype='object')

A continuación, se incluye una breve explicación sobre la información que aporta cada columna.
1. **Description**: Una breve descripción del incidente de ciberseguridad.
2. **Date**: La fecha en que se produjo o descubrió el incidente. Esto ayuda a comprender la cronología del ataque y analizar las tendencias.
3. **Locations**: Una lista de ubicaciones geográficas afectadas por el incidente, normalmente representadas por países o regiones.
4. **Estimated cost**: El impacto financiero estimado del ataque, que puede incluir pérdidas financieras directas, costes de recuperación o rescates pagados.
5. **Victims**: Detalles sobre las víctimas del ataque, normalmente organizaciones o empresas afectadas.
6. **Type of Malware**: El tipo de malware utilizado en el ataque, si se ha identificado.
7. **Threat source**: La fuente o actor detrás de la amenaza, como un grupo de hackers o un actor patrocinado por el Estado.
8. **Industries**: Las industrias afectadas por el ataque, como la automoción, la fabricación, el gobierno, etc. Esto ayuda a identificar los sectores objetivo del ataque.
9. **Impacts**: Los tipos de impactos del ataque, como la interrupción de las operaciones de TI, OT (tecnología operativa) o problemas de privacidad (exfiltración de datos).
10. **References**: Referencias adicionales al incidente, que pueden incluir enlaces a artículos de noticias o informes más detallados del incidente.
11. **Data source link url**: El enlace a la fuente primaria de información sobre el incidente, normalmente un informe o artículo en un sitio web relevante.


In [7]:
df.head(3)

,data_source_link_url,description,date,locations,estimated_cost,victims,type_of_malware,threat_source,industries,impacts,references
0,https://icsstrive.com/incident/operational-imp...,ALPS' North American production operations and...,"September 10, 2023","['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...",No Malware identified,"[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']",OT IT,[{'title': 'Cyber Incident Victim: Alps Alpine...
1,https://icsstrive.com/incident/framework-lapto...,"San Francisco, California-based Framework, the...","January 9, 2024",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No Malware identified,No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...
2,https://icsstrive.com/incident/clothing-provid...,"After suffering a cyberattack December 13, V.F...","December 13, 2023",['United States'],"Cost not known, 35.5 million customers persona...","[{'title': 'V.F. Corporation', 'link_url': 'ht...",No Malware identified,No threat source identified,['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H..."


Separaremos la columna **Date** en componentes de día, mes y año, cada uno de ellos como una columna diferente, convirtiéndola en un formato de fecha.

In [8]:
df['date'] = pd.to_datetime(df['date'])

In [9]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [10]:
df.drop('date', axis = 1, inplace = True)

In [11]:
df.head(2)

,data_source_link_url,description,locations,estimated_cost,victims,type_of_malware,threat_source,industries,impacts,references,year,month,day
0,https://icsstrive.com/incident/operational-imp...,ALPS' North American production operations and...,"['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...",No Malware identified,"[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']",OT IT,[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10
1,https://icsstrive.com/incident/framework-lapto...,"San Francisco, California-based Framework, the...",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No Malware identified,No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9


## Limpieza de datos.

### Manejo de filas duplicadas.
En caso de haber filas duplicadas, deberemos de eliminar estas para que no afecten al análisis posterior.

In [12]:
df.duplicated().sum() # Columnas duplicadas

np.int64(48)

In [13]:
df.drop_duplicates(inplace = True)

In [14]:
df.shape

(1058, 13)

### Manejo de filas nulas.

In [15]:
df.isnull().sum()

data_source_link_url      0
description               0
locations                 0
estimated_cost            0
victims                   0
type_of_malware           0
threat_source             0
industries                0
impacts                 336
references                0
year                      0
month                     0
day                       0
dtype: int64

Como estos valores nulos son pocos, los rellenaremos con la moda en este caso ya que es una variable de tipo objeto.

In [16]:
df['impacts'].fillna(df['impacts'].mode()[0], inplace=True)

C:\Users\ale_m\AppData\Local\Temp\ipykernel_17744\3781718338.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['impacts'].fillna(df['impacts'].mode()[0], inplace=True)


In [17]:
df.shape[0]

1058

### Selección de variables categóricas y continuas.
Para realizar correctamente un ánalisis de datos debemos de distinguir entre las variables categóricas, variables que representan diferentes categorías o grupos, y continuas, variables que pueden tomar cualquier valor dentro de un rango determinado.

In [18]:
v_continuas = []
v_categoricas = []
for i in df.columns:
    if df[i].nunique() > 70 or df[i].dtypes in ['float64', 'int64']:
        v_continuas.append(i)
    else:
        v_categoricas.append(i)

print('Variables continuas: {}'.format(', '.join(v_continuas)))
print('Variables categóricas: {}'.format(', '.join(v_categoricas)))

Variables continuas: data_source_link_url, description, locations, estimated_cost, victims, threat_source, industries, references
Variables categóricas: type_of_malware, impacts, year, month, day


#### Tratamiento de variables categóricas.
Para el posterior modelo de clasificación, necesitamos codificar nuestras variables. Para ello, realizaremos una codificación ordinal que consiste en asignar valores enteros a las categorias basándose en su posición o jerarquía en los datos.

In [19]:
df[v_categoricas].head(3)

,type_of_malware,impacts,year,month,day
0,No Malware identified,OT IT,2023,9,10
1,No Malware identified,IT,2024,1,9
2,No Malware identified,IT,2023,12,13


##### **Impact.**
El proceso consiste en agrupar los valores de la columna **Impacts** en categorías correspondientes. Como algunos valores pueden pertenecer a varias categorías, se realiza una asignación múltiple. Luego, se crean variables dummies para representar esas categorías, marcando con `1` las categorías presentes en cada fila. Esto facilita el análisis de datos en un formato adecuado para modelos y algoritmos.

In [20]:
df['impacts'].unique()

array(['OT IT', 'IT', 'IT Privacy', 'OT', 'Privacy', 'OT Privacy',
       'OT IT Privacy', ' IT', ' OT IT', ' IT Privacy', ' IT Safety',
       ' IT Safety Privacy'], dtype=object)

In [21]:
df['impacts'] = df['impacts'].str.strip() # Eliminar espacios en blanco.

In [22]:
impact_category_mapping = {
    "IT": ["IT", "IT Privacy", "IT Safety", "IT Safety Privacy", "OT IT"],
    "OT": ["OT", "OT Privacy", "OT IT"],
    "Privacy": ["Privacy", "OT Privacy", "IT Privacy"],
}

In [23]:
def map_categories(value, mapping):
    categories = []
    for category, terms in mapping.items():
        if any(term.strip() in value.split() for term in terms):
            categories.append(category)
    return categories

In [24]:
df['impacts'] = df['impacts'].apply(lambda x: ', '.join(set(map_categories(x, impact_category_mapping))))

In [25]:
df_impacts = df['impacts'].str.get_dummies(sep=', ')
df = pd.concat([df, df_impacts], axis=1)

In [26]:
df.head(4)

,data_source_link_url,description,locations,estimated_cost,victims,type_of_malware,threat_source,industries,impacts,references,year,month,day,IT,OT,Privacy
0,https://icsstrive.com/incident/operational-imp...,ALPS' North American production operations and...,"['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...",No Malware identified,"[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']","IT, OT",[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10,1,1,0
1,https://icsstrive.com/incident/framework-lapto...,"San Francisco, California-based Framework, the...",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No Malware identified,No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9,1,0,0
2,https://icsstrive.com/incident/clothing-provid...,"After suffering a cyberattack December 13, V.F...",['United States'],"Cost not known, 35.5 million customers persona...","[{'title': 'V.F. Corporation', 'link_url': 'ht...",No Malware identified,No threat source identified,['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H...",2023,12,13,1,0,0
3,https://icsstrive.com/incident/hackers-accesse...,Toyota Financial Services (TFS) is warning cus...,['Germany'],No cost values disclosed.,"[{'title': 'Toyota Financial Services, subs. o...",No Malware identified,No threat source identified,['Automotive'],"Privacy, IT",[{'title': 'Toyota warns customers of data bre...,2023,12,11,1,0,1


##### **Type of Malware.**
El proceso consiste en convertir las cadenas de la columna, que pueden representar listas de diccionarios, en estructuras adecuadas para extraer la información relevante.

Primero, las cadenas se convierten en listas de diccionarios, manejando posibles errores de formato mediante excepciones para evitar fallos. Cuando la conversión es exitosa, se extraen los valores de las claves `title` y `link_url` de los diccionarios dentro de la lista, asignando valores predeterminados como `No Malware Detected` si no se encuentran datos. Esto permite crear dos nuevas columnas: **Malware title** y **Malware URL**.

In [27]:
df['type_of_malware'].unique()

array(['No Malware identified',
       "[{'title': 'LockBit', 'link_url': 'https://icsstrive.com/malware/lockbit/'}]",
       "[{'title': 'Unknown', 'link_url': 'https://icsstrive.com/malware/unknown-2/'}]",
       "[{'title': 'DDoS Attack', 'link_url': 'https://icsstrive.com/malware/ddos-attack/'}]",
       "[{'title': 'Unknown ransomware variant', 'link_url': 'https://icsstrive.com/malware/unknown-ransomware-variant/'}]",
       "[{'title': 'Ransomware – unknown', 'link_url': 'https://icsstrive.com/malware/ransomware-4/'}]",
       "[{'title': 'Unknown phishing attack', 'link_url': 'https://icsstrive.com/malware/unknown-phishing-attack/'}]",
       "[{'title': 'Ransomware – Unknown group or variant', 'link_url': 'https://icsstrive.com/malware/ransomware-3/'}]",
       "[{'title': 'LightlessCan malware', 'link_url': 'https://icsstrive.com/malware/lightlesscan-malware/'}]",
       "[{'title': 'BlackCat', 'link_url': 'https://icsstrive.com/malware/blackcat/'}]",
       "[{'title': 'Phis

In [28]:
df[df['type_of_malware'] == 'No Malware identified'].shape

(830, 16)

In [29]:
def safely_parse(value):
    try:
        return ast.literal_eval(value) if isinstance(value, str) else value
    except:
        return []

In [30]:
df['type_of_malware'] = df['type_of_malware'].apply(safely_parse)

In [31]:
df['malware_title'] = df['type_of_malware'].apply(lambda x: x[0]['title'] if isinstance(x, list) and x else 'No Malware identified')
df['malware_url'] = df['type_of_malware'].apply(lambda x: x[0]['link_url'] if isinstance(x, list) and x else 'No Malware identified')

In [32]:
df.drop('type_of_malware', axis = 1, inplace = True)

**Malware URL.**

Deberemos de extraer el nombre del dominio de la URL de la nueva columna **Malware URL**.

In [33]:
def extract_domain(url):
    try:
        if not url or "http" not in url:
            return "N/A"
        return urlparse(url).netloc
    except:
        return "N/A"

df['malware_url'] = df['malware_url'].apply(extract_domain)

In [34]:
df['malware_url'].unique()

array(['N/A', 'icsstrive.com'], dtype=object)

**Malware Title.**

In [35]:
df['malware_title'].unique()

array(['No Malware identified', 'LockBit', 'Unknown', 'DDoS Attack',
       'Unknown ransomware variant', 'Ransomware – unknown',
       'Unknown phishing attack', 'Ransomware – Unknown group or variant',
       'LightlessCan malware', 'BlackCat', 'Phishing Attack', 'CLOP',
       'Agenda', 'Ragnar Locker', 'Quantum Ransomware', 'LV', 'PLAY',
       'RansomEXX', '0Ktapus phishing campaign', 'ROADSWEEP',
       'Ransomware [Target: AU Prison]', 'Flame', 'Conti Ransomware',
       'Bitlocker Ransomware', 'NotPetya', 'BlackByte',
       'Ranwomare attack', 'Maui ransomware', 'Snatch Ransomware',
       'Colossus', 'BlackMatter', 'Meteor', 'REvil', 'Avaddon',
       'DarkSide', 'Related to Accellion File Transfer Appliance (FTA)',
       'Phoenix Locker', 'DoppelPaymer', 'Ransomware (target: Gyrodata)',
       'Nefilim Ransomware', 'Ryuk ransomware', 'Netfilim', 'Egregor',
       'Unknown malware attack', 'Mailto', 'Matriex', 'Winnti', 'EKANS',
       'SNAKE', 'unknown ransomware, but unse

In [36]:
malware_mapping = {
    "Ransomware": ["Ransomware"],
    "Spyware": ["Spyware"],
    "Trojan": ["Trojan"],
    "Worm": ["Worm"],
    "No Malware identified": ["No Malware identified", "Unknown malware attack",],
}

In [37]:
def map_malware_title(title):
    for category, titles in malware_mapping.items():
        if title in titles:
            return category
    return "No Malware identified" 

In [38]:
df['malware_title_grouped'] = df['malware_title'].apply(map_malware_title)

In [39]:
label_encoder = LabelEncoder()
df['malware_title_encoded'] = label_encoder.fit_transform(df['malware_title_grouped'])

In [40]:
df.drop('malware_title', axis=1, inplace=True)

df.head(6)

,data_source_link_url,description,locations,estimated_cost,victims,threat_source,industries,impacts,references,year,month,day,IT,OT,Privacy,malware_url,malware_title_grouped,malware_title_encoded
0,https://icsstrive.com/incident/operational-imp...,ALPS' North American production operations and...,"['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...","[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']","IT, OT",[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10,1,1,0,N/A,No Malware identified,0
1,https://icsstrive.com/incident/framework-lapto...,"San Francisco, California-based Framework, the...",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9,1,0,0,N/A,No Malware identified,0
2,https://icsstrive.com/incident/clothing-provid...,"After suffering a cyberattack December 13, V.F...",['United States'],"Cost not known, 35.5 million customers persona...","[{'title': 'V.F. Corporation', 'link_url': 'ht...",No threat source identified,['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H...",2023,12,13,1,0,0,N/A,No Malware identified,0
3,https://icsstrive.com/incident/hackers-accesse...,Toyota Financial Services (TFS) is warning cus...,['Germany'],No cost values disclosed.,"[{'title': 'Toyota Financial Services, subs. o...",No threat source identified,['Automotive'],"Privacy, IT",[{'title': 'Toyota warns customers of data bre...,2023,12,11,1,0,1,N/A,No Malware identified,0
4,https://icsstrive.com/incident/chinese-identif...,Chinese hackers are positioning themselves ins...,['United States'],No cost values disclosed.,"[{'title': 'Unidentified Oil & Gas Pipeline', ...","[{'title': 'Volt Typhoon', 'link_url': 'https:...","['Oil & Gas', 'Water and Waste Water']",IT,[{'title': 'Chinese hackers allegedly target U...,2023,12,11,1,0,0,N/A,No Malware identified,0
5,https://icsstrive.com/incident/blue-waters-bot...,Blue Waters Products Limited is the latest loc...,['Trinidad & Tobago'],unknown,"[{'title': 'Blue Waters Products Limited', 'li...","[{'title': 'LockBit', 'link_url': 'https://ics...",['Water and Waste Water'],IT,[{'title': 'LockBit3 takes responsibility for ...,2023,12,9,1,0,0,N/A,No Malware identified,0


### Tratamiento de variables continuas.
Para el modelo de clasificación debemos de pasar todas las columnas a tipo `int`. Por ello, debemos de ver si realizar como en el caso anterior una codificación ordinal o agruparlos.

In [41]:
df[v_continuas].head(2)

,data_source_link_url,description,locations,estimated_cost,victims,threat_source,industries,references
0,https://icsstrive.com/incident/operational-imp...,ALPS' North American production operations and...,"['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...","[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']",[{'title': 'Cyber Incident Victim: Alps Alpine...
1,https://icsstrive.com/incident/framework-lapto...,"San Francisco, California-based Framework, the...",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No threat source identified,['Technology'],[{'title': 'Laptop Maker Framework Says Custom...


In [42]:
v_categoricas_cont = list(df[v_continuas].dtypes[df[v_continuas].dtypes == 'object'].index)

In [43]:
df[v_categoricas_cont].nunique()

data_source_link_url    1058
description              860
locations                135
estimated_cost           355
victims                  790
threat_source             97
industries                91
references               820
dtype: int64

#### **Data Source Link URL.**
Deberemos de extraer el nombre del dominio de la URL.

In [44]:
def extract_domain(url):
    try:
        return urlparse(url).netloc
    except:
        return None

df['data_source_link_url'] = df['data_source_link_url'].apply(extract_domain)

In [45]:
df['data_source_link_url'] = df['data_source_link_url'].str.replace('www.', '', regex=False)

In [46]:
df.head(3)

,data_source_link_url,description,locations,estimated_cost,victims,threat_source,industries,impacts,references,year,month,day,IT,OT,Privacy,malware_url,malware_title_grouped,malware_title_encoded
0,icsstrive.com,ALPS' North American production operations and...,"['Japan', 'United States']",Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...","[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']","IT, OT",[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10,1,1,0,N/A,No Malware identified,0
1,icsstrive.com,"San Francisco, California-based Framework, the...",['United States'],Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9,1,0,0,N/A,No Malware identified,0
2,icsstrive.com,"After suffering a cyberattack December 13, V.F...",['United States'],"Cost not known, 35.5 million customers persona...","[{'title': 'V.F. Corporation', 'link_url': 'ht...",No threat source identified,['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H...",2023,12,13,1,0,0,N/A,No Malware identified,0


#### **Locations.**
Para realizar una evaluación más efectiva de los países en nuestros datos, es conveniente agruparlos por continentes. Esto es especialmente relevante porque en otros archivos CSV hay registros de ataques provenientes de diferentes países, distribuidos en distintos continentes.

Para facilitar este análisis, procederemos a crear seis nuevas columnas en nuestro conjunto de datos, cada una representando un continente específico. Estas columnas indicarán con un valor de 1 si el país pertenece al continente correspondiente y 0 si no es así, es decir, mediante una asignación de valores binarios.

In [47]:
df['locations'].unique()

array(["['Japan', 'United States']", "['United States']", "['Germany']",
       "['Trinidad & Tobago']", "['Curaçao']",
       "['Australia', 'New Zealand']", "['Australia', 'United States']",
       "['Italy', 'United Kingdom']", "['France']", "['Japan']",
       "['Netherlands', 'United States']", "['United Kingdom']",
       "['Australia']", "['Germany', 'United States']", "['India']",
       "['Russia']", "['Philippines']", "['Canada']", "['Taiwan']",
       "['Spain']", "['Italy']", "['Bermuda']", "['Israel']",
       "['New Zealand']", "['France', 'Netherlands']",
       "['Canada', 'United States']", "['Sri Lanka']",
       "['Germany', 'Netherlands']", "['Poland']", "['Denmark']",
       "['Portugal']", "['China']", "['Kenya']", "['Norway']",
       "['Sweden', 'United Kingdom']", "['Ireland']",
       "['Germany', 'Global']", "['Belgium', 'Netherlands']",
       "['Netherlands']", "['Switzerland']", '[]', "['Belgium']",
       "['France', 'Global']", "['Sweden']", "['Croatia']

Vemos que existen los caracteres `[` y `]`, así como las comillas, y los eliminaremos para dejar las cadenas de texto más limpias y legibles.

In [48]:
df['locations'] = df['locations'].str.strip("[]").str.replace("'", "")

Creamos un diccionario que relaciona países con continentes para mejorar la eficiencia en el proceso de detección.

In [49]:
country_to_continent = {
    'Japan': 'Asia', 'United States': 'North America', 'Germany': 'Europe',
    'Trinidad & Tobago': 'North America', 'Curaçao': 'South America',
    'Australia': 'Oceania', 'New Zealand': 'Oceania', 'Italy': 'Europe',
    'United Kingdom': 'Europe', 'France': 'Europe', 'Netherlands': 'Europe',
    'Philippines': 'Asia', 'Canada': 'North America', 'Taiwan': 'Asia',
    'Spain': 'Europe', 'Bermuda': 'North America', 'Israel': 'Asia',
    'Sri Lanka': 'Asia', 'Poland': 'Europe', 'Denmark': 'Europe', 'Portugal': 'Europe',
    'China': 'Asia', 'Kenya': 'Africa', 'Norway': 'Europe', 'Sweden': 'Europe',
    'Ireland': 'Europe', 'Switzerland': 'Europe', 'Luxembourg': 'Europe', 'Malaysia': 'Asia',
    'Greece': 'Europe', 'Finland': 'Europe', 'Chile': 'South America', 'Iran': 'Asia',
    'South Africa': 'Africa', 'Brazil': 'South America', 'Nigeria': 'Africa',
    'Singapore': 'Asia', 'Guadeloupe': 'North America', 'Estonia': 'Europe',
    'Ecuador': 'South America', 'Saudi Arabia': 'Asia', 'Turkey': 'Asia',
    'Mexico': 'North America', 'Argentina': 'South America', 'South Korea': 'Asia',
    'Romania': 'Europe', 'Belgium': 'Europe', 'Malta': 'Europe', 'Belarus': 'Europe',
    'Russia': 'Europe', 'Qatar': 'Asia', 'Finland': 'Europe', 'Vietnam': 'Asia',
}

In [50]:
def assign_continent(country):
    # Primero, intentamos asignar usando el diccionario
    continent = country_to_continent.get(country, None)
    # Si no está en el diccionario, usamos coco.convert
    if not continent:
        try:
            continent = coco.convert(names=country, to='continent')
        except:
            continent = None  # Si coco no puede asignarlo, dejamos None
    return continent


In [51]:
def assign_continents_to_countries(countries):
    country_list = countries.split(', ') 
    continents_set = set()  # Usar un conjunto para evitar duplicados
    for country in country_list:
        continent = assign_continent(country)  # Usamos la función para asignar continente
        if continent:
            if continent in ['North America', 'South America']:
                continents_set.add('America')
            elif continent in ['Africa', 'Asia', 'Europe', 'Oceania']:
                continents_set.add(continent)
            else:
                continents_set.add('Not Found')  # Para cualquier caso no identificado
    return ', '.join(continents_set)  

In [52]:
df['continents'] = df['locations'].apply(assign_continents_to_countries)

Global not found in regex
 not found in regex
Global not found in regex
Global not found in regex
 not found in regex
 not found in regex
 not found in regex
 not found in regex
Asia-Pacific not found in regex
 not found in regex
Global not found in regex
 not found in regex
Global not found in regex
 not found in regex
 not found in regex
 not found in regex
 not found in regex
Europe not found in regex
South America not found in regex
Africa not found in regex
Europe not found in regex
Middle East not found in regex
Global not found in regex
Global not found in regex
Asia-Pacific not found in regex
Europe not found in regex
Global not found in regex
Global not found in regex
Global not found in regex
Global not found in regex
Global not found in regex
Global not found in regex
Europe not found in regex
Global not found in regex
Asia-Pacific not found in regex
Global not found in regex
Middle East not found in regex
Global not found in regex
Global not found in regex
Europe not found 

In [53]:
df.drop('locations', axis=1, inplace=True)

In [54]:
continent_dummies = df['continents'].str.get_dummies(sep=', ')

In [55]:
df = pd.concat([df, continent_dummies], axis=1)

df.head(7)

,data_source_link_url,description,estimated_cost,victims,threat_source,industries,impacts,references,year,month,...,malware_url,malware_title_grouped,malware_title_encoded,continents,Africa,America,Asia,Europe,Not Found,Oceania
0,icsstrive.com,ALPS' North American production operations and...,Partially impacted North American production a...,"[{'title': 'Alps Alpine Group', 'link_url': 'h...","[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']","IT, OT",[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,...,N/A,No Malware identified,0,"Asia, America",0,1,1,0,0,0
1,icsstrive.com,"San Francisco, California-based Framework, the...",Unknown as it seems PII was stolen,"[{'title': 'Framework', 'link_url': 'https://i...",No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,...,N/A,No Malware identified,0,America,0,1,0,0,0,0
2,icsstrive.com,"After suffering a cyberattack December 13, V.F...","Cost not known, 35.5 million customers persona...","[{'title': 'V.F. Corporation', 'link_url': 'ht...",No threat source identified,['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H...",2023,12,...,N/A,No Malware identified,0,America,0,1,0,0,0,0
3,icsstrive.com,Toyota Financial Services (TFS) is warning cus...,No cost values disclosed.,"[{'title': 'Toyota Financial Services, subs. o...",No threat source identified,['Automotive'],"Privacy, IT",[{'title': 'Toyota warns customers of data bre...,2023,12,...,N/A,No Malware identified,0,Europe,0,0,0,1,0,0
4,icsstrive.com,Chinese hackers are positioning themselves ins...,No cost values disclosed.,"[{'title': 'Unidentified Oil & Gas Pipeline', ...","[{'title': 'Volt Typhoon', 'link_url': 'https:...","['Oil & Gas', 'Water and Waste Water']",IT,[{'title': 'Chinese hackers allegedly target U...,2023,12,...,N/A,No Malware identified,0,America,0,1,0,0,0,0
5,icsstrive.com,Blue Waters Products Limited is the latest loc...,unknown,"[{'title': 'Blue Waters Products Limited', 'li...","[{'title': 'LockBit', 'link_url': 'https://ics...",['Water and Waste Water'],IT,[{'title': 'LockBit3 takes responsibility for ...,2023,12,...,N/A,No Malware identified,0,America,0,1,0,0,0,0
6,icsstrive.com,A regulatory agency in Florida that oversees t...,No cost values disclosed.,[{'title': 'St. Johns River Water Management D...,"[{'title': 'Cyber Av3ngers', 'link_url': 'http...",['Water and Waste Water'],IT,[{'title': 'Florida water agency latest to con...,2023,12,...,N/A,No Malware identified,0,America,0,1,0,0,0,0


#### **Victims.**
Es necesario categorizar las víctimas presentes en cada registro. Se procesará la columna y cada entrada se limpiará y se convertirá en una lista de víctimas, extrayendo solo los nombres o titulos relevantes. Posteriormente, las victimas se categorizaran en las siguientes categorias:

- `Corporation/Group` : para nombres que incluyan términnos como 'Group' o 'Inc'.
- `Corporation` : para nombres que incluyan 'Corporation'.
- `Company` : para nombres que incluyan 'Company'.
- `Other` : para aquellos casos que no encajen en ninguna de las categorías anteriores.

Teniendo al final, una nueva columna llamada **categorized_victims**, mejorando la interpretación y análisis de los tipos involucrados en los incidentes

In [56]:
df['victims'].unique()

array(["[{'title': 'Alps Alpine Group', 'link_url': 'https://icsstrive.com/victim/alps-alpine-group/'}]",
       "[{'title': 'Framework', 'link_url': 'https://icsstrive.com/victim/framework/'}]",
       "[{'title': 'V.F. Corporation', 'link_url': 'https://icsstrive.com/victim/v-f-corporation/'}]",
       "[{'title': 'Toyota Financial Services, subs. of Toyota Motor Corp', 'link_url': 'https://icsstrive.com/victim/toyota-financial-services-subs-of-toyota-motor-corp/'}]",
       "[{'title': 'Unidentified Oil & Gas Pipeline', 'link_url': 'https://icsstrive.com/victim/unidentified-oil-gas-pipeline/'}, {'title': 'Hawaii Water Utility', 'link_url': 'https://icsstrive.com/victim/hawaii-water-utility/'}]",
       "[{'title': 'Blue Waters Products Limited', 'link_url': 'https://icsstrive.com/victim/blue-waters-products-limited/'}]",
       "[{'title': 'St. Johns River Water Management District, Florida', 'link_url': 'https://icsstrive.com/victim/st-johns-river-water-management-district-florida/

In [57]:
df[df['victims'].isna()].shape

(0, 24)

Limpaimos la columna eliminando los corchetes y comillas no necesarias

In [58]:
df['victims'] = df['victims'].str.strip("[]").str.replace("'", "")

In [63]:
def clean_victims(victim_string):
    if pd.isna(victim_string):
        return "N/A"

    try:
        victim_list = eval(victim_string)
        victim_names = [victim['title'] for victim in victim_list if 'title' in victim]
        categorized_victims = categorize_victims(victim_names)
        return ', '.join(categorized_victims) if categorized_victims else "N/A"
    except Exception:
        return "N/A"

In [64]:
def categorize_victims(victim_names):
    categories = []

    for name in victim_names:
        if 'Group' in name or 'Inc.' in name:
            categories.append('Corporation/Group')
        elif 'Corporation' in name:
            categories.append('Corporation')
        elif 'Company' in name:
            categories.append('Company')
        else:
            categories.append('Other')

    return list(set(categories))

In [65]:
df['categorized_victims'] = df['victims'].apply(clean_victims)

In [66]:
df.drop('victims', axis=1, inplace=True)

df.head(8)

,data_source_link_url,description,estimated_cost,threat_source,industries,impacts,references,year,month,day,...,malware_title_grouped,malware_title_encoded,continents,Africa,America,Asia,Europe,Not Found,Oceania,categorized_victims
0,icsstrive.com,ALPS' North American production operations and...,Partially impacted North American production a...,"[{'title': 'BlackByte', 'link_url': 'https://i...","['Automotive', 'Manufacturing']","IT, OT",[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10,...,No Malware identified,0,"Asia, America",0,1,1,0,0,0,N/A
1,icsstrive.com,"San Francisco, California-based Framework, the...",Unknown as it seems PII was stolen,No threat source identified,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9,...,No Malware identified,0,America,0,1,0,0,0,0,N/A
2,icsstrive.com,"After suffering a cyberattack December 13, V.F...","Cost not known, 35.5 million customers persona...",No threat source identified,['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H...",2023,12,13,...,No Malware identified,0,America,0,1,0,0,0,0,N/A
3,icsstrive.com,Toyota Financial Services (TFS) is warning cus...,No cost values disclosed.,No threat source identified,['Automotive'],"Privacy, IT",[{'title': 'Toyota warns customers of data bre...,2023,12,11,...,No Malware identified,0,Europe,0,0,0,1,0,0,N/A
4,icsstrive.com,Chinese hackers are positioning themselves ins...,No cost values disclosed.,"[{'title': 'Volt Typhoon', 'link_url': 'https:...","['Oil & Gas', 'Water and Waste Water']",IT,[{'title': 'Chinese hackers allegedly target U...,2023,12,11,...,No Malware identified,0,America,0,1,0,0,0,0,N/A
5,icsstrive.com,Blue Waters Products Limited is the latest loc...,unknown,"[{'title': 'LockBit', 'link_url': 'https://ics...",['Water and Waste Water'],IT,[{'title': 'LockBit3 takes responsibility for ...,2023,12,9,...,No Malware identified,0,America,0,1,0,0,0,0,N/A
6,icsstrive.com,A regulatory agency in Florida that oversees t...,No cost values disclosed.,"[{'title': 'Cyber Av3ngers', 'link_url': 'http...",['Water and Waste Water'],IT,[{'title': 'Florida water agency latest to con...,2023,12,6,...,No Malware identified,0,America,0,1,0,0,0,0,N/A
7,icsstrive.com,Aqualectra Utilities witnessed a breach of its...,No cost values disclosed.,"[{'title': 'Akira Ransomware Gang', 'link_url'...","['Energy (Includes Power and Utilities)', 'Wat...","Privacy, IT",[{'title': 'Detected: Aqualectra Utility falls...,2023,12,6,...,No Malware identified,0,America,0,1,0,0,0,0,N/A


##### **Threat Sources.**
Convertiremos las cadenas de texto de la columna **threat_source** en listas de diccionarios. Para manejar posibles errores de formato, utilizaremos una fuincion para convertir a un objeto adecuado. Si no hay valores será manejado como N/A. Al terminar la conversion, extraeremos los valores de `title` y `link_url` para crear dos nuevas columnas: **Threat source title** y **Threat source link URL**.

In [67]:
df['threat_source'].unique()

array(["[{'title': 'BlackByte', 'link_url': 'https://icsstrive.com/threat-actor/blackbyte/'}]",
       'No threat source identified',
       "[{'title': 'Volt Typhoon', 'link_url': 'https://icsstrive.com/threat-actor/volt-typhoon/'}]",
       "[{'title': 'LockBit', 'link_url': 'https://icsstrive.com/threat-actor/lockbit/'}]",
       "[{'title': 'Cyber Av3ngers', 'link_url': 'https://icsstrive.com/threat-actor/cyber-av3ngers/'}]",
       "[{'title': 'Akira Ransomware Gang', 'link_url': 'https://icsstrive.com/threat-actor/akira-ransomware-gang/'}]",
       "[{'title': 'Hunters International', 'link_url': 'https://icsstrive.com/threat-actor/hunters-international/'}]",
       "[{'title': 'Play, aka PlayCrypt', 'link_url': 'https://icsstrive.com/threat-actor/play/'}]",
       "[{'title': 'Medusa', 'link_url': 'https://icsstrive.com/threat-actor/medusa/'}]",
       "[{'title': 'BlackCat / ALPHV', 'link_url': 'https://icsstrive.com/threat-actor/blackcat/'}]",
       "[{'title': 'Cerber ransom

In [68]:
df[df['threat_source'] == 'No threat source identified'].shape

(771, 24)

Funcion para limpiar las cadenas de texto y convertirlas en listas de diccionarios.

In [69]:
def safely_parse(value):
    try:
        return ast.literal_eval(value) if isinstance(value, str) else value
    except:
        return []

In [70]:
df['threat_source'] = df['threat_source'].apply(safely_parse)

Extraemos los títulos y enlaces.

In [71]:
def clean_threat_source(threat_source_list):
    if not isinstance(threat_source_list, list) or not threat_source_list:
        return "N/A", "N/A"

    threat_titles = []
    threat_links = []

    for threat in threat_source_list:
        if 'title' in threat:
            title = threat['title']
            if 'unknown' in title.lower():
                title = 'Unknown'  # Reemplazar por "Unknown" si el título contiene "unknown"
            threat_titles.append(title)

        if 'link_url' in threat:
            link_url = threat['link_url']
            domain = extract_domain(link_url)  # Extraemos el dominio de la URL
            threat_links.append(domain)

    return (
        ', '.join(threat_titles) if threat_titles else "N/A",
        ', '.join(threat_links) if threat_links else "N/A"
    )

In [72]:
df[['Threat source title', 'Threat source link URL']] = df['threat_source'].apply(
    lambda x: pd.Series(clean_threat_source(x))
)

In [73]:
df.drop('threat_source', axis=1, inplace=True)

df.head(9)

,data_source_link_url,description,estimated_cost,industries,impacts,references,year,month,day,IT,...,continents,Africa,America,Asia,Europe,Not Found,Oceania,categorized_victims,Threat source title,Threat source link URL
0,icsstrive.com,ALPS' North American production operations and...,Partially impacted North American production a...,"['Automotive', 'Manufacturing']","IT, OT",[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10,1,...,"Asia, America",0,1,1,0,0,0,N/A,BlackByte,icsstrive.com
1,icsstrive.com,"San Francisco, California-based Framework, the...",Unknown as it seems PII was stolen,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9,1,...,America,0,1,0,0,0,0,N/A,N/A,N/A
2,icsstrive.com,"After suffering a cyberattack December 13, V.F...","Cost not known, 35.5 million customers persona...",['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H...",2023,12,13,1,...,America,0,1,0,0,0,0,N/A,N/A,N/A
3,icsstrive.com,Toyota Financial Services (TFS) is warning cus...,No cost values disclosed.,['Automotive'],"Privacy, IT",[{'title': 'Toyota warns customers of data bre...,2023,12,11,1,...,Europe,0,0,0,1,0,0,N/A,N/A,N/A
4,icsstrive.com,Chinese hackers are positioning themselves ins...,No cost values disclosed.,"['Oil & Gas', 'Water and Waste Water']",IT,[{'title': 'Chinese hackers allegedly target U...,2023,12,11,1,...,America,0,1,0,0,0,0,N/A,Volt Typhoon,icsstrive.com
5,icsstrive.com,Blue Waters Products Limited is the latest loc...,unknown,['Water and Waste Water'],IT,[{'title': 'LockBit3 takes responsibility for ...,2023,12,9,1,...,America,0,1,0,0,0,0,N/A,LockBit,icsstrive.com
6,icsstrive.com,A regulatory agency in Florida that oversees t...,No cost values disclosed.,['Water and Waste Water'],IT,[{'title': 'Florida water agency latest to con...,2023,12,6,1,...,America,0,1,0,0,0,0,N/A,Cyber Av3ngers,icsstrive.com
7,icsstrive.com,Aqualectra Utilities witnessed a breach of its...,No cost values disclosed.,"['Energy (Includes Power and Utilities)', 'Wat...","Privacy, IT",[{'title': 'Detected: Aqualectra Utility falls...,2023,12,6,1,...,America,0,1,0,0,0,0,N/A,Akira Ransomware Gang,icsstrive.com
8,icsstrive.com,Japanese automobile manufacturer Nissan announ...,internal systems disrupted for several days,"['Automotive', 'Other']","Privacy, IT",[{'title': 'A CYBER ATTACK HIT NISSAN OCEANIA'...,2023,12,5,1,...,Oceania,0,0,0,0,0,1,N/A,N/A,N/A


##### **Threat Source URL.**

Extraemos el dominio de la URL de la fuente de amenaza y lo almacenamos en la columna **Threat source link URL**.

In [74]:
def extract_domain(url):
    try:
        if not url or "http" not in url:
            return "N/A"  # Si no se encuentra una URL válida, retornar "N/A"
        return urlparse(url).netloc
    except:
        return "N/A" 

In [75]:
df['Threat source link URL'] = df['Threat source link URL'].apply(extract_domain)

In [76]:
df['Threat source link URL'].unique()

array(['N/A'], dtype=object)

##### **Threat Source Title.**

Normalizamos los títulos con "unknown" a "Unknown" y luego los categorizamos según su relevancia y tipo.

In [77]:
def categorize_threat_source(title):
    # Definimos algunas categorías
    categories = {
        "Unknown": ["Unknown", "Unknown source", "No source available"],
        "Government": ["Government", "State", "National Agency"],
        "Private": ["Private", "Private Sector", "Corporation"]
    }

    for category, titles in categories.items():
        if any(keyword in title for keyword in titles):
            return category
    return "Other"  # Si no coincide con ninguna categoría definida

In [78]:
df['Threat source title Categorized'] = df['Threat source title'].apply(categorize_threat_source)

In [79]:
df['Threat source title Categorized'].unique()

df.head(10)

,data_source_link_url,description,estimated_cost,industries,impacts,references,year,month,day,IT,...,Africa,America,Asia,Europe,Not Found,Oceania,categorized_victims,Threat source title,Threat source link URL,Threat source title Categorized
0,icsstrive.com,ALPS' North American production operations and...,Partially impacted North American production a...,"['Automotive', 'Manufacturing']","IT, OT",[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10,1,...,0,1,1,0,0,0,N/A,BlackByte,N/A,Other
1,icsstrive.com,"San Francisco, California-based Framework, the...",Unknown as it seems PII was stolen,['Technology'],IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9,1,...,0,1,0,0,0,0,N/A,N/A,N/A,Other
2,icsstrive.com,"After suffering a cyberattack December 13, V.F...","Cost not known, 35.5 million customers persona...",['Manufacturing'],IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H...",2023,12,13,1,...,0,1,0,0,0,0,N/A,N/A,N/A,Other
3,icsstrive.com,Toyota Financial Services (TFS) is warning cus...,No cost values disclosed.,['Automotive'],"Privacy, IT",[{'title': 'Toyota warns customers of data bre...,2023,12,11,1,...,0,0,0,1,0,0,N/A,N/A,N/A,Other
4,icsstrive.com,Chinese hackers are positioning themselves ins...,No cost values disclosed.,"['Oil & Gas', 'Water and Waste Water']",IT,[{'title': 'Chinese hackers allegedly target U...,2023,12,11,1,...,0,1,0,0,0,0,N/A,Volt Typhoon,N/A,Other
5,icsstrive.com,Blue Waters Products Limited is the latest loc...,unknown,['Water and Waste Water'],IT,[{'title': 'LockBit3 takes responsibility for ...,2023,12,9,1,...,0,1,0,0,0,0,N/A,LockBit,N/A,Other
6,icsstrive.com,A regulatory agency in Florida that oversees t...,No cost values disclosed.,['Water and Waste Water'],IT,[{'title': 'Florida water agency latest to con...,2023,12,6,1,...,0,1,0,0,0,0,N/A,Cyber Av3ngers,N/A,Other
7,icsstrive.com,Aqualectra Utilities witnessed a breach of its...,No cost values disclosed.,"['Energy (Includes Power and Utilities)', 'Wat...","Privacy, IT",[{'title': 'Detected: Aqualectra Utility falls...,2023,12,6,1,...,0,1,0,0,0,0,N/A,Akira Ransomware Gang,N/A,Other
8,icsstrive.com,Japanese automobile manufacturer Nissan announ...,internal systems disrupted for several days,"['Automotive', 'Other']","Privacy, IT",[{'title': 'A CYBER ATTACK HIT NISSAN OCEANIA'...,2023,12,5,1,...,0,0,0,0,0,1,N/A,N/A,N/A,Other
9,icsstrive.com,Austal USA confirmed that it suffered a cybera...,No cost values disclosed.,"['Transportation (Includes Logistics, Shipping...",IT,[{'title': 'Hunters International Ransomware V...,2023,12,2,1,...,0,1,0,0,0,1,N/A,Hunters International,N/A,Other


##### **Industries.**

Para facilitar el análisis, crearemos categorias para agruparlas. Por tal motivo, crearemos una nueva columna llamada **industries_grouped**, que contendrá el nombre de la categoría de la industria a la que pertenece cada fila. Primero, se limpiarán los datos manejando posibles valores nulos o vacios. Luego, se utilizará un conjunto de categorías predifinidas tales como; `Manufacturing`, `Technology`, `Healthcare`, entre otras.

In [99]:
def clean_industries(industries_string):
    if pd.isna(industries_string) or industries_string == "[]":
        return "N/A"  # Si está vacío o es una lista vacía, devolver "N/A"
    
    try:
        # Eliminar los corchetes y comillas
        industries_string = industries_string.strip("[]").replace("'", "")
        
      
        industries_list = industries_string.split(', ') if industries_string else []

        # Si la lista tiene elementos, devolverlos como una cadena separada por comas
        if industries_list:
            return ', '.join(industries_list)
        else:
            return "N/A"
    except Exception:
        return "N/A"  # Si ocurre un error, devolver "N/A"

In [121]:
    def group_industries(industries_string):
        manufacturing = ["Aerospace", "Automotive", "Chemicals", "Metals and Mining", "Steel", "Food and Beverage", "Pulp and Paper", "Manufacturing"]
        technology = ["Software", "Cloud Provider", "Communications", "Technology"]
        healthcare = ["Biotech", "Pharmaceutical", "Healthcare"]
        transportation = ["Transportation (Includes Logistics, Shipping, Maritime, Rail, Trucking"]
        energy = ["Oil and Gas", "Water and Waste Water"]
        government_defense = ["Government", "Defense"]
        business_services = ["Business"]
        public_sector = ["Other"]
        finance = ["Banking", "Insurance"]
    
        industries_list = industries_string.split(', ')
    
        if any(industry in industries_list for industry in manufacturing):
            return "Manufacturing"
        elif any(industry in industries_list for industry in technology):
            return "Technology"
        elif any(industry in industries_list for industry in healthcare):
            return "Healthcare"
        elif any(industry in industries_list for industry in transportation):
            return "Transportation and Logistics"
        elif any(industry in industries_list for industry in energy):
            return "Energy and Utilities"
        elif any(industry in industries_list for industry in government_defense):
            return "Government and Defense"
        elif any(industry in industries_list for industry in business_services):
            return "Business Services"
        elif any(industry in industries_list for industry in public_sector):
            return "Public Sector and Non-Profit"
        elif any(industry in industries_list for industry in finance):
            return "Finance"
        else:
            return "Other"  # Para casos no definidos

In [ ]:
if 'industries' in df.columns:
    df['industries'] = df['industries'].apply(clean_industries)
    df['industries_grouped'] = df['industries'].apply(group_industries)

In [ ]:
df.drop('industries', axis=1, inplace=True)

df.head(11)

,data_source_link_url,description,estimated_cost,impacts,references,year,month,day,IT,OT,...,America,Asia,Europe,Not Found,Oceania,categorized_victims,Threat source title,Threat source link URL,Threat source title Categorized,industries_grouped
0,icsstrive.com,ALPS' North American production operations and...,Partially impacted North American production a...,"IT, OT",[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10,1,1,...,1,1,0,0,0,N/A,BlackByte,N/A,Other,None
1,icsstrive.com,"San Francisco, California-based Framework, the...",Unknown as it seems PII was stolen,IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9,1,0,...,1,0,0,0,0,N/A,N/A,N/A,Other,None
2,icsstrive.com,"After suffering a cyberattack December 13, V.F...","Cost not known, 35.5 million customers persona...",IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H...",2023,12,13,1,0,...,1,0,0,0,0,N/A,N/A,N/A,Other,None
3,icsstrive.com,Toyota Financial Services (TFS) is warning cus...,No cost values disclosed.,"Privacy, IT",[{'title': 'Toyota warns customers of data bre...,2023,12,11,1,0,...,0,0,1,0,0,N/A,N/A,N/A,Other,None
4,icsstrive.com,Chinese hackers are positioning themselves ins...,No cost values disclosed.,IT,[{'title': 'Chinese hackers allegedly target U...,2023,12,11,1,0,...,1,0,0,0,0,N/A,Volt Typhoon,N/A,Other,None
5,icsstrive.com,Blue Waters Products Limited is the latest loc...,unknown,IT,[{'title': 'LockBit3 takes responsibility for ...,2023,12,9,1,0,...,1,0,0,0,0,N/A,LockBit,N/A,Other,None
6,icsstrive.com,A regulatory agency in Florida that oversees t...,No cost values disclosed.,IT,[{'title': 'Florida water agency latest to con...,2023,12,6,1,0,...,1,0,0,0,0,N/A,Cyber Av3ngers,N/A,Other,None
7,icsstrive.com,Aqualectra Utilities witnessed a breach of its...,No cost values disclosed.,"Privacy, IT",[{'title': 'Detected: Aqualectra Utility falls...,2023,12,6,1,0,...,1,0,0,0,0,N/A,Akira Ransomware Gang,N/A,Other,None
8,icsstrive.com,Japanese automobile manufacturer Nissan announ...,internal systems disrupted for several days,"Privacy, IT",[{'title': 'A CYBER ATTACK HIT NISSAN OCEANIA'...,2023,12,5,1,0,...,0,0,0,0,1,N/A,N/A,N/A,Other,None
9,icsstrive.com,Austal USA confirmed that it suffered a cybera...,No cost values disclosed.,IT,[{'title': 'Hunters International Ransomware V...,2023,12,2,1,0,...,1,0,0,0,1,N/A,Hunters International,N/A,Other,None


##### **Estimated Cost.**

Para aquellos casos en los que el valor del costo este ausente o sea indeterminado "N/A", se reemplazaran con la moda de los valores presentes en la columna de costo estimado en funcion a la industria a la que pertenencen. Además, se creará una nueva columna **cost_category**, que represnetará la categoría del costo estimado con las siguientes categorías: 

- `Low` : para costos bajos
- `Medium` : para costos medios
- `High` : para costos altos
- `Very High` : para valores más elevados

In [ ]:
df['estimated_cost'].unique()

In [ ]:
df[df['estimated_cost'] == 'N/A'].shape

In [137]:
import re

def clean_estimated_cost(cost_string):
    if pd.isna(cost_string):
        return "N/A"
    cost_string = str(cost_string).strip()
    if any(word in cost_string.lower() for word in ['no cost values disclosed', 'unknown', 'unsure']):
        return "N/A"

    if re.search(r'\d+\+?\s*(days|hours|minutes|seconds|weeks)', cost_string.lower()):
        return "N/A"

    match = re.search(r'\$\s*(\d+(?:[.,]\d{1,2})?)(\s?[KkMm]?)', cost_string)

    if match:
        number = match.group(1).replace(',', '')
        unit = match.group(2).upper().strip()

        try:
            number = float(number)
        except ValueError:
            return "N/A"
        if unit == 'M':
            return number * 1_000_000
        elif unit == 'K':
            return number * 1_000
        else:
            return number

    match_range = re.search(r'\$\s*(\d+(?:[.,]\d{1,2})?)\s*[-to]+\s*(\d+(?:[.,]\d{1,2})?)\s*(million|M|K|k)?', cost_string)
    if match_range:
        low_value = match_range.group(1).replace(',', '')
        high_value = match_range.group(2).replace(',', '')
        unit = match_range.group(3).upper() if match_range.group(3) else None

        try:
            low_value = float(low_value)
            high_value = float(high_value)
        except ValueError:
            return "N/A"

        if unit in ['M', 'million']:
            low_value *= 1_000_000
            high_value *= 1_000_000
        elif unit in ['K', 'k']:
            low_value *= 1_000
            high_value *= 1_000

        return f"{int(low_value)}-{int(high_value)}"

    # Si no se encuentra ningún valor numérico válido, retornar N/A
    return "N/A"

In [138]:
df['estimated_cost'] = df['estimated_cost'].apply(clean_estimated_cost)

In [139]:
def fill_cost_based_on_industry_with_any_value(row):
    if row['estimated_cost'] == 'N/A':
        industry_group = row['industries_grouped']
        
        # Filtrar los costos válidos de la misma industria
        valid_industry_costs = df[df['industries_grouped'] == industry_group]['estimated_cost']
        valid_industry_costs = pd.to_numeric(valid_industry_costs, errors='coerce').dropna()

        # Si hay costos válidos para la industria, devolver el primero encontrado
        if not valid_industry_costs.empty:
            return valid_industry_costs.iloc[0]  # Tomar el primer valor válido
        return 'N/A'  # Si no hay valores válidos, devolver "N/A"
    else:
        return row['estimated_cost']

In [140]:
df['estimated_cost'] = df.apply(fill_cost_based_on_industry_with_any_value, axis=1)

In [141]:
def categorize_estimated_cost(value):
    if value == "N/A" or pd.isna(value):
        return "Unknown"

    try:
        if isinstance(value, float):
            cost_value = value
        elif isinstance(value, str) and '-' in value:
            low, high = map(float, value.split('-'))
            cost_value = (low + high) / 2
        else:
            cost_value = float(value)
    except ValueError:
        return "Unknown"

    if cost_value < 100_000:
        return "Low"
    elif 100_000 <= cost_value < 1_000_000:
        return "Medium"
    elif 1_000_000 <= cost_value < 10_000_000:
        return "High"
    else:
        return "Very High"

In [142]:
df['cost_category'] = df['estimated_cost'].apply(categorize_estimated_cost)


df.head(12)

,data_source_link_url,description,estimated_cost,impacts,references,year,month,day,IT,OT,...,Asia,Europe,Not Found,Oceania,categorized_victims,Threat source title,Threat source link URL,Threat source title Categorized,industries_grouped,cost_category
0,icsstrive.com,ALPS' North American production operations and...,N/A,"IT, OT",[{'title': 'Cyber Incident Victim: Alps Alpine...,2023,9,10,1,1,...,1,0,0,0,N/A,BlackByte,N/A,Other,None,Unknown
1,icsstrive.com,"San Francisco, California-based Framework, the...",N/A,IT,[{'title': 'Laptop Maker Framework Says Custom...,2024,1,9,1,0,...,0,0,0,0,N/A,N/A,N/A,Other,None,Unknown
2,icsstrive.com,"After suffering a cyberattack December 13, V.F...",N/A,IT,"[{'title': 'Clothing Brand, V.F. Corp., Hack H...",2023,12,13,1,0,...,0,0,0,0,N/A,N/A,N/A,Other,None,Unknown
3,icsstrive.com,Toyota Financial Services (TFS) is warning cus...,N/A,"Privacy, IT",[{'title': 'Toyota warns customers of data bre...,2023,12,11,1,0,...,0,1,0,0,N/A,N/A,N/A,Other,None,Unknown
4,icsstrive.com,Chinese hackers are positioning themselves ins...,N/A,IT,[{'title': 'Chinese hackers allegedly target U...,2023,12,11,1,0,...,0,0,0,0,N/A,Volt Typhoon,N/A,Other,None,Unknown
5,icsstrive.com,Blue Waters Products Limited is the latest loc...,N/A,IT,[{'title': 'LockBit3 takes responsibility for ...,2023,12,9,1,0,...,0,0,0,0,N/A,LockBit,N/A,Other,None,Unknown
6,icsstrive.com,A regulatory agency in Florida that oversees t...,N/A,IT,[{'title': 'Florida water agency latest to con...,2023,12,6,1,0,...,0,0,0,0,N/A,Cyber Av3ngers,N/A,Other,None,Unknown
7,icsstrive.com,Aqualectra Utilities witnessed a breach of its...,N/A,"Privacy, IT",[{'title': 'Detected: Aqualectra Utility falls...,2023,12,6,1,0,...,0,0,0,0,N/A,Akira Ransomware Gang,N/A,Other,None,Unknown
8,icsstrive.com,Japanese automobile manufacturer Nissan announ...,N/A,"Privacy, IT",[{'title': 'A CYBER ATTACK HIT NISSAN OCEANIA'...,2023,12,5,1,0,...,0,0,0,1,N/A,N/A,N/A,Other,None,Unknown
9,icsstrive.com,Austal USA confirmed that it suffered a cybera...,N/A,IT,[{'title': 'Hunters International Ransomware V...,2023,12,2,1,0,...,0,0,0,1,N/A,Hunters International,N/A,Other,None,Unknown
